#Install libraries

In [1]:
!pip install -q google-adk google-genai python-dotenv


#Configure Gemini and sanity check

In [2]:
import os
from google import genai

os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

client = genai.Client(api_key=GEMINI_API_KEY)

resp = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents="Say hello as an e-commerce shopping assistant.",
)
print(resp.text)


Hello! I'm your e-commerce shopping assistant. How can I help you find the perfect product today?



#Product catalog in SQLite

In [3]:
import sqlite3, os

DB_PATH = "ecommerce.db"
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS products (
    id        INTEGER PRIMARY KEY AUTOINCREMENT,
    sku       TEXT,
    name      TEXT,
    category  TEXT,
    price     REAL,
    currency  TEXT,
    description TEXT
)
""")
conn.commit()

# Seed a few example products if table is empty
cur.execute("SELECT COUNT(*) FROM products")
count = cur.fetchone()[0]

if count == 0:
    products = [
        ("SKU-1001", "Nimbus Runner", "running shoes", 110.0, "USD", "Neutral cushioned running shoe for daily training."),
        ("SKU-1002", "TrailClimb Pro", "trail shoes", 130.0, "USD", "Aggressive trail shoe with rock plate and deep lugs."),
        ("SKU-2001", "AeroFit Tee", "tops", 35.0, "USD", "Lightweight moisture-wicking running t-shirt."),
        ("SKU-3001", "HydroFlex Bottle", "accessories", 18.0, "USD", "Soft flask with bite valve for long runs."),
    ]
    cur.executemany(
        "INSERT INTO products (sku, name, category, price, currency, description) VALUES (?, ?, ?, ?, ?, ?)",
        products,
    )
    conn.commit()
    print("Seeded sample products.")
else:
    print(f"Products already present: {count}")


Seeded sample products.


#Product search tool

In [4]:
from google.adk.tools import FunctionTool

def search_products(query: str) -> list[dict]:
    """
    Search products by name or category using a simple LIKE query.
    Returns a list of dicts with basic product info.
    """
    pattern = f"%{query.lower()}%"
    cur.execute(
        """
        SELECT sku, name, category, price, currency, description
        FROM products
        WHERE LOWER(name) LIKE ? OR LOWER(category) LIKE ?
        """,
        (pattern, pattern),
    )
    rows = cur.fetchall()
    results = []
    for sku, name, category, price, currency, desc in rows:
        results.append(
            {
                "sku": sku,
                "name": name,
                "category": category,
                "price": price,
                "currency": currency,
                "description": desc,
            }
        )
    return results

search_products_tool = FunctionTool(func=search_products)


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [5]:
print(search_products("running"))


[{'sku': 'SKU-1001', 'name': 'Nimbus Runner', 'category': 'running shoes', 'price': 110.0, 'currency': 'USD', 'description': 'Neutral cushioned running shoe for daily training.'}]


#E-commerce assistant agent

In [6]:
from google.adk.agents import LlmAgent

ecom_agent = LlmAgent(
    name="ecommerce_assistant",
    model="gemini-2.0-flash-001",
    instruction=(
        "You are an e-commerce shopping assistant.\n"
        "- The user will describe what they are looking for.\n"
        "- Use the search_products tool to find matching items in the catalog.\n"
        "- Then recommend 1–3 good options with SKU, name, price, and a short explanation.\n"
        "- Do not invent products that are not in the catalog."
    ),
    tools=[search_products_tool],
)


#Runner and session

In [7]:
from google.adk.runners import InMemoryRunner

APP_NAME = "ecommerce_app"
USER_ID = "demo-ecom-user"

runner = InMemoryRunner(agent=ecom_agent, app_name=APP_NAME)

session = await runner.session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
)

print("Session ID:", session.id)


Session ID: be23f0b6-3735-4c29-9620-d7ed820e21f1


#Helper to ask e-commerce questions

In [8]:
from google.genai.types import Content, Part

def ask_ecom_agent(message: str):
    user_msg = Content(
        role="user",
        parts=[Part.from_text(text=message)],
    )

    print("=== USER ===")
    print(message)
    print("\n=== ASSISTANT ===")

    for event in runner.run(
        user_id=session.user_id,
        session_id=session.id,
        new_message=user_msg,
    ):
        if hasattr(event, "is_final_response") and event.is_final_response():
            if event.content:
                for part in event.content.parts:
                    text = getattr(part, "text", None)
                    if text:
                        print(text.strip())

# test it
ask_ecom_agent("I'm looking for running shoes for daily training under $120.")


=== USER ===
I'm looking for running shoes for daily training under $120.

=== ASSISTANT ===


I found one option that fits your needs:

*   **SKU-1001**, **Nimbus Runner**, **$110**, Neutral cushioned running shoe for daily training.
